In [332]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

In [333]:
train_sounds = pd.read_csv(r'C:\Users\rlee3104\beholdAI\train_behold.csv')

In [334]:
train_sounds.head()

,contrast,dissimilarity,homogeneity,ASM,energy,correlation,subarachnoid
0,3477.947917,24.919588,0.592016,0.312388,0.558917,0.695715,1
1,3477.947917,24.919588,0.592016,0.312388,0.558917,0.695715,0
2,3378.826601,19.703506,0.724975,0.479028,0.692118,0.759936,1
3,3065.715828,20.438770,0.699484,0.448966,0.670049,0.707154,1
4,5983.737551,32.912221,0.635906,0.358061,0.598382,0.500977,0


In [336]:
train_label = train_sounds['subarachnoid']

In [337]:
count = train_sounds['subarachnoid'].value_counts() 

In [338]:
count

0    5584
1    2958
Name: subarachnoid, dtype: int64

In [366]:
5584/2958

1.8877620013522651

In [339]:
train_sounds.drop(['subarachnoid'],inplace=True,axis=1)

In [340]:
from xgboost import XGBClassifier

In [368]:
from sklearn.model_selection import train_test_split, cross_val_score

X_train, X_test, y_train, y_test = train_test_split(
    train_sounds, train_label, test_size=0.2, random_state=42
)

In [379]:
params = {'colsample_bytree': 0.8,'learning_rate': 0.001, 'max_depth': 8, 'n_estimators': 1000, 'subsample': 1,'objective':'binary:logistic','eval_metric':'logloss','scale_pos_weight':1.2}

In [380]:
import xgboost as xgb
dt =xgdmat=xgb.DMatrix(X_train,y_train)
dte = xgdmat=xgb.DMatrix(X_test,y_test)

In [384]:
model = xgb.train(params, dt, 2800, [(dt, "train"),(dte, "valid")], verbose_eval=200)

[0]	train-logloss:0.692996	valid-logloss:0.693053
[200]	train-logloss:0.667	valid-logloss:0.677334
[400]	train-logloss:0.646584	valid-logloss:0.666688
[600]	train-logloss:0.630158	valid-logloss:0.659239
[800]	train-logloss:0.615982	valid-logloss:0.654203
[1000]	train-logloss:0.603491	valid-logloss:0.650466
[1200]	train-logloss:0.592556	valid-logloss:0.647862
[1400]	train-logloss:0.583601	valid-logloss:0.646231
[1600]	train-logloss:0.575507	valid-logloss:0.645221
[1800]	train-logloss:0.568166	valid-logloss:0.644758
[2000]	train-logloss:0.562004	valid-logloss:0.644657
[2200]	train-logloss:0.557158	valid-logloss:0.644455
[2400]	train-logloss:0.552872	valid-logloss:0.644079
[2600]	train-logloss:0.548709	valid-logloss:0.643757
[2799]	train-logloss:0.544882	valid-logloss:0.643635


In [385]:
from termcolor import colored
#from sklearn.cross_validation import KFold

from sklearn.metrics import confusion_matrix

In [386]:
y_pred1 = model.predict(dte)
cm = confusion_matrix(y_test, (y_pred1>0.5))
print(colored('The Confusion Matrix is: ', 'red'),'\n', cm)
predict_accuracy_on_test_set = (cm[0,0] + cm[1,1])/(cm[0,0] + cm[1,1]+cm[1,0] + cm[0,1])
print(colored('The Accuracy on Test Set is: ', 'blue'), colored(predict_accuracy_on_test_set, 'blue'))

The Confusion Matrix is:  
 [[1031   85]
 [ 548   45]]
The Accuracy on Test Set is:  0.6296079578700995


In [387]:
test_sounds = pd.read_csv(r'C:\Users\rlee3104\beholdAI\test_behold.csv')

In [388]:
test_sounds.head()

,contrast,dissimilarity,homogeneity,ASM,energy,correlation
0,3073.033410,18.406631,0.693477,0.433153,0.658144,0.734190
1,4619.377160,28.053735,0.636671,0.370178,0.608423,0.543405
2,4428.263592,28.837652,0.558793,0.281347,0.530421,0.626958
3,4486.181148,28.757876,0.554076,0.273923,0.523376,0.654095
4,5390.584921,28.406822,0.704247,0.357729,0.598105,0.719922


In [389]:
dtest = xgdmat=xgb.DMatrix(test_sounds)

In [390]:
y_epidural = model.predict(dtest)

In [391]:
y_epidural

array([0.5082298 , 0.3352754 , 0.4597022 , ..., 0.39774388, 0.39720154,
       0.45640457], dtype=float32)

In [392]:
my_submission = pd.DataFrame({'subarachnoid': y_epidural.ravel()})
my_submission.to_csv('subarachnoid.csv', index=False)